<a href="https://colab.research.google.com/github/bhanuprasadtudy/advancedadvertisingstatistics/blob/master/Geolocating_Twitter_Data_from_Profile_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install geostring
import geostring as geo
import glob
import os
import shutil
import json
import csv
import zipfile

     |████████████████████████████████| 194kB 2.9MB/s 
     |████████████████████████████████| 245kB 37.7MB/s 
World index created.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
tweetzipfiles = glob.glob('/content/drive/My Drive/Colab Notebooks/Twitter analysis/*.zip')

In [0]:
tweetzipfiles

['/content/drive/My Drive/Colab Notebooks/Twitter analysis/@McDonalds.zip',
 '/content/drive/My Drive/Colab Notebooks/Twitter analysis/@kfc.zip',
 '/content/drive/My Drive/Colab Notebooks/Twitter analysis/@wendys.zip']

In [0]:
location = []
tweet =[]
for tweetzipfile in tweetzipfiles:
  zips= zipfile.ZipFile(tweetzipfile)
  for i, obj in enumerate(zips.infolist()): 
    tweet_json = json.load(zips.open(obj)) 
    if (geo.resolve(tweet_json['user']['location'])) is not None:
      tweet.append(tweet_json['user']['id'])
      location.append(geo.resolve(tweet_json['user']['location']))

In [0]:
import pandas as pd
locationwise = pd.DataFrame(location)
locationwise

,resolved_city,resolved_subcountry,resolved_country
0,winnipeg,manitoba,canada
1,,california,united states
2,cincinnati,ohio,united states
3,,new jersey,united states
4,san antonio,texas,united states
...,...,...,...
10018,lawrence,indiana?kansas?massachusetts?new jersey,united states
10019,brossard,quebec,canada
10020,asheville,north carolina,united states
10021,,florida,united states


In [0]:
df_new = pd.DataFrame(locationwise)
df_new['id'] = tweet
USAtweets = df_new['resolved_country']=="united states"
USAtweets_final = df_new[USAtweets]
USAtweets_final.head()

,resolved_city,resolved_subcountry,resolved_country,id
1,,california,united states,270252242
2,cincinnati,ohio,united states,3340073541
3,,new jersey,united states,3244131477
4,san antonio,texas,united states,2989059767
6,,tennessee,united states,818229702427111429


In [0]:
statewisetweets= USAtweets_final.groupby('resolved_subcountry')['id'].agg(['count']).to_dict()

In [0]:
dic = {}
for i in USAtweets_final['resolved_subcountry']:
    dic[i] = [{USAtweets_final['id'][j]} for j in USAtweets_final[USAtweets_final['resolved_subcountry']==i].index]

In [0]:
print('Collected Tweets for', len(dic), 'states')
print('Colorado has', len(dic['colorado']), 'tweets')
some_id = dic['colorado'][0]
print('The ID of the first Colorado tweet is:',some_id)

Collected Tweets for 288 states
Colorado has 167 tweets
The ID of the first Colorado tweet is: {3222622209}


In [0]:
top3states = USAtweets_final.groupby(['resolved_subcountry']).size()
top3states.nlargest(4)

resolved_subcountry
california    613
texas         495
              452
new york      452
dtype: int64